### OpenAI CartPole https://gymnasium.farama.org/environments/classic_control/cart_pole/

## Install dependancies

In [ ]:
# %% [markdown]
# # CartPole Reinforcement Learning in Colab
#
# This notebook demonstrates how to train an agent to balance the CartPole using Deep Q-Learning.

# %%
!pip install gymnasium 
!pip install tensorflow 

  Obtaining dependency information for gymnasium from https://files.pythonhosted.org/packages/f9/68/2bdc7b46b5f543dd865575f9d19716866bdb76e50dd33b71ed1a3dd8bb42/gymnasium-1.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for numpy>=1.21.0 from https://files.pythonhosted.org/packages/98/89/0c93baaf0094bdaaaa0536fe61a27b1dce8a505fa262a865ec142208cfe9/numpy-2.2.5-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------------------- ------------ 41.0/60.8 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 816.2 kB/s eta 0:00:00
  Obtaining dependency information for cloudpickle>=1.2.0 from https://files.pythonhosted.org/packages/7e/e8/64c37fadfc2816a7701fa8a6ed8d87327c7d54eacfbfb6edab14a2f2be75/cloudpickle-3.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for farama-notifications>=0.0.1


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import numpy as np
import gymnasium as gym
import tensorflow as tf
from tensorflow import keras
from collections import deque
import random
import matplotlib.pyplot as plt

# 1. Ініціалізація середовища


env = gym.make("CartPole-v1", render_mode="human")  

Для візуалізації у вікні

або

env = gym.make("CartPole-v1", render_mode="rgb_array")  

Для отримання кадру як масиву numpy

In [ ]:
env = gym.make('CartPole-v1', render_mode="rgb_array")
state_size = env.observation_space.shape[0]  # 4 змінні: позиція, швидкість, кут, кутова швидкість
action_size = env.action_space.n  # 2 дії: рух вліво (0) або вправо (1)
print(f"State size: {state_size}, Action size: {action_size}")

State size: 4, Action size: 2


CartPole-v1 — середовище, де агент керує візком, щоб утримувати стрижень у вертикальному положенні.

Стан (state): 4 числа:

Позиція візка (x),

Швидкість візка (v),

Кут стрижня (θ),

Кутова швидкість стрижня (ω).

Дії (actions): 0 (ліворуч) або 1 (праворуч).

## 2. Побудова нейромережі


In [ ]:
# Deep Q-Network model
def build_model(state_size, action_size):
    model = keras.Sequential([
        keras.layers.Dense(24, input_dim=state_size, activation='relu'),
        keras.layers.Dense(24, activation='relu'),
        keras.layers.Dense(action_size, activation='linear')
    ])
    model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=0.001))
    return model

Архітектура нейромережі для DQN у задачі CartPole обрана не випадково — вона ґрунтується на поєднанні теоретичних принципів глибокого навчання та практичного досвіду. Розберімо чому саме такий дизайн:

1. **Вхідний шар (4 нейрони)**

*   **Чому 4?**

Вхідний шар відповідає розмірності стану середовища CartPole:
[позиція візка, швидкість візка, кут стрижня, кутова швидкість стрижня].
Кожен нейрон приймає одну з цих змінних.

*   **Чому немає додаткової обробки?**

Дані вже нормалізовані (на відміну, наприклад, від зображень), тому не потрібні складні шари (на кшталт згорткових).

**2. Приховані шари (2 шари по 24 нейрони з ReLU)**

* **Чому 2 шари?**

**Перший** шар виділяє прості ознаки (наприклад, залежність кута від швидкості).

**Другий** шар комбінує їх у складніші паттерни (наприклад, "якщо стрижень відхиляється вліво і рухається швидко, потрібно їхати вліво").
Глибші шари (3+) рідко дають виграш для таких простих задач.

* **Чому 24 нейрони?**

Емпіричне правило для DQN:

Занадто мало нейронів (наприклад, 8) → мережа не зможе навчитися складним залежностям.

Занадто багато (наприклад, 128) → ризик перенавчання або повільного навчання.
**24 — компроміс між швидкістю та якістю для CartPole.**

* **Чому ReLU?**

* * **Переваги ReLU:**

* * * Нелінійність (дозволяє навчатися складним функціям).

* * * Швидкі обчислення (порівняно з Tanh/Sigmoid).

* * * Уникає проблеми "зникаючих градієнтів" (як у Sigmoid).

* * Для Q-навчання критично мати нелінійність, щоб апроксимувати складні Q-функції.

**3. Вихідний шар** (2 нейрони з лінійною активацією)

* **Чому 2 нейрони?**

Кількість дій у CartPole — 2 (ліворуч/праворуч). Кожен нейрон виводить Q-значення для відповідної дії.

* **Чому лінійна активація?**

Q-значення можуть бути будь-якими числами (від -∞ до +∞), тому лінійна активація (без обмежень) ідеальна для регресії.

**4. Функція втрат (MSE)**

* **Чому MSE?**

Завдання — передбачити Q-значення (регресія). MSE карає за великі відхилення від цільових значень (Bellman targets):

$$
loss = (Q_{predicted} - (reward + γ * max(Q_{target}))^2
$$

Альтернативи (наприклад, Huber loss) менш чутливі до викидів, але MSE часто працює стабільніше для DQN.

**5. Оптимізатор** (Adam з lr=0.001)

* **Чому Adam?**

* * Адаптує learning rate для кожного параметра (краще, ніж SGD).

* * Стабілізує навчання за рахунок моментуму.

* * Працює краще за RMSprop для DQN у більшості досліджень.

* **Чому lr=0.001?**

* * Занадто великий (наприклад, 0.01) → навчання нестабільне.

* * Занадто малий (наприклад, 0.0001) → повільне навчання.

* * 0.001 — стандартне значення для багатьох RL-задач.

**Емпіричне підтвердження**

Така архітектура стала де-факто стандартом після публікацій DeepMind (наприклад, Human-level control through deep reinforcement learning):

* Для простіших задач (як CartPole) достатньо 2-х прихованих шарів.

* Більше нейронів (24-64) дає кращу апроксимацію Q-функції, але потребує більше даних.

* ReLU + Adam — найефективніше поєднання для швидкого навчання.

**Що буде, якщо змінити архітектуру?**

Зміна	| Ефект

1 прихований шар |	Мережа може не навчитися складним стратегіям (наприклад, балансування при великій швидкості).

Більше 2 шарів | Ризик перенавчання або повільного навчання без реального виграшу.

Sigmoid замість ReLU | Повільне навчання через "зникаючі градієнти".
Більше нейронів (наприклад, 128)	Може покращити якість, але вимагає більше даних і часу.

Менше нейронів (наприклад, 8) |	Мережа не зможе знайти оптимальну стратегію.

Для CartPole запропонована архітектура — це оптимальний баланс між простотою та ефективністю. Для складніших задач (наприклад, Atari) використовують більші мережі (згорткові + повнозв’язні шари).

**Математичний опис архітектури DQN для CartPole**

Нейромережа реалізує функцію апроксимації $Q(s,a;\theta)$, де:

* $s \in \mathbb{R}^4$ - стан середовища
* $a \in \{0,1\}$ - дія
* $\theta$ - параметри мережі


**1. Вхідний шар**

Приймає вектор стану:
$$ \mathbf{x}^{(0)} = \mathbf{s} $$

**2. Перший прихований шар**
\begin{align*}
    \mathbf{z}^{(1)} &= \mathbf{W}^{(1)}\mathbf{x}^{(0)} + \mathbf{b}^{(1)} \\
    & \text{де } \mathbf{W}^{(1)} \in \mathbb{R}^{24 \times 4}, \mathbf{b}^{(1)} \in \mathbb{R}^{24} \\
    \mathbf{x}^{(1)} &= \text{ReLU}(\mathbf{z}^{(1)}) = \max(0, \mathbf{z}^{(1)})
\end{align*}

**3. Другий прихований шар**
\begin{align*}
    \mathbf{z}^{(2)} &= \mathbf{W}^{(2)}\mathbf{x}^{(1)} + \mathbf{b}^{(2)} \\
    & \text{де } \mathbf{W}^{(2)} \in \mathbb{R}^{24 \times 24}, \mathbf{b}^{(2)} \in \mathbb{R}^{24} \\
    \mathbf{x}^{(2)} &= \text{ReLU}(\mathbf{z}^{(2)})
\end{align*}

**4. Вихідний шар**
\begin{align*}
    \mathbf{Q}(s,\cdot;\theta) &= \mathbf{W}^{(3)}\mathbf{x}^{(2)} + \mathbf{b}^{(3)} \\
    & \text{де } \mathbf{W}^{(3)} \in \mathbb{R}^{2 \times 24}, \mathbf{b}^{(3)} \in \mathbb{R}^{2}
\end{align*}

**Функція втрат**
Для одного прикладу $(s,a,r,s',\text{done})$:
$$
\mathcal{L}(\theta) =
\begin{cases}
(r - Q(s,a;\theta))^2 & \text{якщо done = True} \\
(r + \gamma \max_{a'} Q(s',a';\theta^-) - Q(s,a;\theta))^2 & \text{інакше}
\end{cases}
$$
де $\gamma = 0.95$ - коефіцієнт дисконтування, $\theta^-$ - параметри цільової мережі.

**Повний forward pass**
Для батчу станів $\mathbf{S} \in \mathbb{R}^{\text{batch\_size} \times 4}$:
$$
\mathbf{Q}(\mathbf{S};\theta) = f_{\theta}^{(3)} \circ f_{\theta}^{(2)} \circ f_{\theta}^{(1)}(\mathbf{S})
$$
де:
\begin{align*}
    f_{\theta}^{(1)}(\mathbf{x}) &= \text{ReLU}(\mathbf{W}^{(1)}\mathbf{x} + \mathbf{b}^{(1)}) \\
    f_{\theta}^{(2)}(\mathbf{x}) &= \text{ReLU}(\mathbf{W}^{(2)}\mathbf{x} + \mathbf{b}^{(2)}) \\
    f_{\theta}^{(3)}(\mathbf{x}) &= \mathbf{W}^{(3)}\mathbf{x} + \mathbf{b}^{(3)}
\end{align*}

**Оновлення параметрів**
Використовується оптимізатор Adam:
$$
\theta_{t+1} = \theta_t - \alpha_t \hat{\mathbf{m}}_t / (\sqrt{\hat{\mathbf{v}}_t} + \epsilon)
$$
де $\alpha_t = 0.001$, $\hat{\mathbf{m}}_t$ і $\hat{\mathbf{v}}_t$ - коректені оцінки першого та другого моментів.

Ця архітектура є компромісом між:

 * **Ємністю моделі** (здатністю апроксимувати складну Q-функцію)

 * **Швидкістю навчання** (обмежена кількість параметрів)

 * **Стабільністю** (використання ReLU та Adam для уникнення проблем з градієнтами)

**3. Алгоритм DQN агента**

In [ ]:
# %%
# DQN Agent
class DQNAgent:

    # Ініціалізація:

    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000) # Булфер досвіду (experience replay)
        self.gamma = 0.95    # Коефіцієнт дисконтування майбутніх нагород
        self.epsilon = 1.0    # Початкова ймовірність випадкової дії (exploration)
        self.epsilon_min = 0.01 # Мінімальний epsilon
        self.epsilon_decay = 0.995 # Швидкість зменшення epsilon
        self.model = build_model(state_size, action_size)
        self.target_model = build_model(state_size, action_size)
        self.update_target_model()

      # Experience Replay: Зберігає попередні досвіди (state, action, reward, next_state, done) для навчання.
      # Gamma: Визначає важливість майбутніх нагород (0.95 = агент планує на 20 кроків уперед).
      # Epsilon-Greedy: Спочатку агент досліджує (випадкові дії), потім все більше експлуатує знання.

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    # Зберігання досвіду (remember):

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

      # Додає кортеж (state, action, reward, next_state, done) до буфера memory.

    # Вибір дії (act):

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state, verbose=0)
        return np.argmax(act_values[0])

     # Якщо epsilon = 0.1, то в 10% випадків дія випадкова, в 90% — обрана на основі Q-значень.

    # Навчання на досвіді (replay):

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size) # Вибірка 32 випадкових досвідів
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state, verbose=0) # Поточні прогнози Q-значень
            if done:
                target[0][action] = reward # Якщо епізод закінчено, Q = reward
            else:
                t = self.target_model.predict(next_state, verbose=0)
                target[0][action] = reward + self.gamma * np.amax(t[0]) # Bellman equation
            self.model.fit(state, target, epochs=1, verbose=0) # Корекція ваг
        # Політика epsilon-greedy
        # Епсилон зменшується після кожного навчання:
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
          # Результат:
            # Спочатку агент досліджує (випадкові дії), потім переходить до експлуатації знань.

        # Bellman Equation:
        #Q(s,a) = reward + γ * max(Q(s',a'))
        #де s' — наступний стан, a' — найкраща дія в ньому.

        #Target Model: Допомагає стабілізувати навчання, використовуючи окрему мережу для прогнозування Q(s',a').

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

4. Цикл навчання

In [ ]:
# %%
# Training parameters
EPISODES = 700
BATCH_SIZE = 32

agent = DQNAgent(state_size, action_size)
done = False
scores = []

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# %%
# Training loop
for e in range(EPISODES):
    # Скидання середовища:
    state, _ = env.reset()
    state = np.reshape(state, [1, state_size])
    total_reward = 0

    for time in range(500):
        # Uncomment to render (slows down training)
        env.render()
        #Крок агента:
          #Обрати дію (act),
          #виконати її в середовищі (env.step).
        # Після навчання агент виконує дії на основі Q-значень без досліджень (epsilon = 0):
        action = agent.act(state)
        next_state, reward, done, _, _ = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward

        if done:
            # Оновлення цільової мережі:
            agent.update_target_model()
            print(f"episode: {e}/{EPISODES}, score: {time}, e: {agent.epsilon:.2f}")
            scores.append(time)
            break

    # Навчання:
      # Зберегти досвід (remember).
    if len(agent.memory) > BATCH_SIZE:
        agent.replay(BATCH_SIZE)

# %%
# Plot training progress
plt.plot(scores)
plt.title('Training Progress')
plt.xlabel('Episode')
plt.ylabel('Score')
plt.show()

# %%
# Test the trained agent
test_episodes = 10
for e in range(test_episodes):
    state, _ = env.reset()
    state = np.reshape(state, [1, state_size])
    total_reward = 0

    for time in range(500):
        env.render()  # Uncomment to visualize
        action = agent.act(state)
        next_state, reward, done, _, _ = env.step(action)
        total_reward += reward
        state = np.reshape(next_state, [1, state_size])

        if done:
            print(f"episode: {e}/{test_episodes}, score: {time}")
            break


# %%
# Plot training progress
plt.plot(scores)
plt.title('Training Progress')
plt.xlabel('Episode')
plt.ylabel('Score')
plt.show()

# %%
# Test the trained agent
test_episodes = 10
for e in range(test_episodes):
    state, _ = env.reset()
    state = np.reshape(state, [1, state_size])
    total_reward = 0

    for time in range(500):
        env.render()  # Uncomment to visualize
        action = agent.act(state)
        next_state, reward, done, _, _ = env.step(action)
        total_reward += reward
        state = np.reshape(next_state, [1, state_size])

        if done:
            print(f"episode: {e}/{test_episodes}, score: {time}")
            break



episode: 0/700, score: 13, e: 1.00
episode: 1/700, score: 22, e: 1.00
episode: 2/700, score: 17, e: 0.99
episode: 3/700, score: 16, e: 0.99
episode: 4/700, score: 14, e: 0.99
episode: 5/700, score: 16, e: 0.98
episode: 6/700, score: 11, e: 0.98
episode: 7/700, score: 16, e: 0.97
episode: 8/700, score: 21, e: 0.97
episode: 9/700, score: 12, e: 0.96
episode: 10/700, score: 9, e: 0.96
episode: 11/700, score: 13, e: 0.95
episode: 12/700, score: 15, e: 0.95
episode: 13/700, score: 33, e: 0.94
episode: 14/700, score: 13, e: 0.94
episode: 15/700, score: 9, e: 0.93
episode: 16/700, score: 12, e: 0.93
episode: 17/700, score: 29, e: 0.92
episode: 18/700, score: 20, e: 0.92
episode: 19/700, score: 10, e: 0.91
episode: 20/700, score: 12, e: 0.91
episode: 21/700, score: 16, e: 0.90
episode: 22/700, score: 50, e: 0.90
episode: 23/700, score: 36, e: 0.90
episode: 24/700, score: 28, e: 0.89
episode: 25/700, score: 32, e: 0.89
episode: 26/700, score: 38, e: 0.88
episode: 27/700, score: 36, e: 0.88
epis

In [ ]:
# %%
# Save the model
agent.model.save("cartpole-dqn.keras")  # Keras 3.x
loaded_model = keras.models.load_model("cartpole-dqn.keras")